### Project - Airline AI Assistant
We'll now bring together what we've learned to make an AI Customer Support assistant for an Airline

In [2]:
import os, json
from dotenv import load_dotenv
from openai import OpenAI
import gradio as gr

In [3]:
load_dotenv(override=True)
google_api_key = os.getenv('GOOGLE_API_KEY')

if google_api_key:
    print(f"Google API Key exists and begins {google_api_key[:2]}")
else:
    print("Google API Key not set (and this is optional)")

Google API Key exists and begins AI


In [6]:
gemini_url = "https://generativelanguage.googleapis.com/v1beta/openai/"
gemini = OpenAI(api_key=google_api_key, base_url=gemini_url)

MODEL = "gemini-2.5-flash-lite"

In [5]:
system_message = """
You are a helpful assistant, for an airline called FlightAI.
Give short and courteous responses no more than one sentence long.
Always be accurate, if you don't know the answer, just say you don't know.
"""

In [8]:
def chat(message, history):
    updated_history = [{'role': message['role'], 'content': message['content']} for message in history]

    messages = [{'role': 'system', 'content': system_message}]
    messages.extend(updated_history)
    messages.append({'role': 'user', 'content': message})

    response = gemini.chat.completions.create(model=MODEL, messages=messages)

    return response.choices[0].message.content

gr.ChatInterface(fn=chat, type="messages").launch()


* Running on local URL:  http://127.0.0.1:7887
* To create a public link, set `share=True` in `launch()`.


#### Tools

- we see that it does not have some flight related information
- to augment this unknown information, we can add tools

Tools are an incredibly powerful feature provided by the frontier LLMs.

With tools, you can write a function, and have the LLM call that function as part of its response.

Sounds almost spooky.. we're giving it the power to run code on our machine?

Well, kinda.

##### Step 1: Setting up the tool

In [9]:
# Let's start by making a useful function

ticket_prices = {"london": "$799", "paris": "$899", "tokyo": "$1400", "berlin": "$499"}

def get_ticket_price(destination_city):
    print(f"Tool called for city {destination_city}")
    price = ticket_prices.get(destination_city.lower(), "Unknown ticket price")
    return f"The price of a ticket to {destination_city} is {price}"

In [10]:
get_ticket_price("London")

Tool called for city London


'The price of a ticket to London is $799'

In [11]:
# There's a particular dictionary structure that's required to describe our function:

price_function = {
    "name": "get_ticket_price",
    "description": "Get the price of a return ticket to the destination city.",
    "parameters": {
        "type": "object",
        "properties": {
            "destination_city": {
                "type": "string",
                "description": "The city that the customer wants to travel to",
            },
        },
        "required": ["destination_city"],
        "additionalProperties": False
    }
}

In [12]:
# And this is included in a list of tools:

tools = [{"type": "function", "function": price_function}]

## this is what is sent to LLM
tools

[{'type': 'function',
  'function': {'name': 'get_ticket_price',
   'description': 'Get the price of a return ticket to the destination city.',
   'parameters': {'type': 'object',
    'properties': {'destination_city': {'type': 'string',
      'description': 'The city that the customer wants to travel to'}},
    'required': ['destination_city'],
    'additionalProperties': False}}}]

##### Step 2: Getting LLM to use our Tool
There's some fiddly stuff to allow LLM "to call our tool"

What we actually do is give the LLM the opportunity to inform us that it wants us to run the tool.

Here's how the new chat function looks:

In [32]:
def chat(message, history):
    updated_history = [{'role': message['role'], 'content': message['content']} for message in history]

    messages = [{'role': 'system', 'content': system_message}]
    messages.extend(updated_history)
    messages.append({'role': 'user', 'content': message})

    response = gemini.chat.completions.create(model=MODEL, messages=messages, tools=tools)

    # if the finish reason is tool calls then, we need to call the tool
    if response.choices[0].finish_reason=="tool_calls":
        message = response.choices[0].message
        response = handle_tool_call(message)
        messages.append(message)
        messages.append(response)

        for message in messages:
            print(message)
        response = gemini.chat.completions.create(model=MODEL, messages=messages)
    
    return response.choices[0].message.content

In [33]:
# We have to write that function handle_tool_call:

def handle_tool_call(message):
    tool_call = message.tool_calls[0]
    if tool_call.function.name == "get_ticket_price":
        arguments = json.loads(tool_call.function.arguments)
        city = arguments.get('destination_city')
        price_details = get_ticket_price(city)
        response = {
            "role": "tool",
            "content": price_details,
            "tool_call_id": tool_call.id
        }
    return response

In [34]:
gr.ChatInterface(fn=chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7895
* To create a public link, set `share=True` in `launch()`.


Tool called for city London
{'role': 'system', 'content': "\nYou are a helpful assistant, for an airline called FlightAI.\nGive short and courteous responses no more than one sentence long.\nAlways be accurate, if you don't know the answer, just say you don't know.\n"}
{'role': 'user', 'content': 'Hi'}
{'role': 'assistant', 'content': 'Hello! How can I help you today?'}
{'role': 'user', 'content': 'I want to travel'}
{'role': 'assistant', 'content': 'Great, where would you like to go?'}
{'role': 'user', 'content': 'London'}
ChatCompletionMessage(content=None, refusal=None, role='assistant', annotations=None, audio=None, function_call=None, tool_calls=[ChatCompletionMessageFunctionToolCall(id='function-call-13109184504573719208', function=Function(arguments='{"destination_city":"London"}', name='get_ticket_price'), type='function')])
{'role': 'tool', 'content': 'The price of a ticket to London is $799', 'tool_call_id': 'function-call-13109184504573719208'}


Traceback (most recent call last):
  File "/Users/srinivas/Documents/Others/My_projects/Python/DSAIML/LLM_eng/.venv/lib/python3.11/site-packages/gradio/queueing.py", line 759, in process_events
    response = await route_utils.call_process_api(
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/srinivas/Documents/Others/My_projects/Python/DSAIML/LLM_eng/.venv/lib/python3.11/site-packages/gradio/route_utils.py", line 354, in call_process_api
    output = await app.get_blocks().process_api(
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/srinivas/Documents/Others/My_projects/Python/DSAIML/LLM_eng/.venv/lib/python3.11/site-packages/gradio/blocks.py", line 2116, in process_api
    result = await self.call_function(
             ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/srinivas/Documents/Others/My_projects/Python/DSAIML/LLM_eng/.venv/lib/python3.11/site-packages/gradio/blocks.py", line 1621, in call_function
    prediction = await fn(*processed_input)
        

#### Let's make a couple of improvements
Handling multiple tool calls in 1 response

Handling multiple tool calls 1 after another

In [39]:
def chat(message, history):
    updated_history = [{'role': message['role'], 'content': message['content']} for message in history]

    messages = [{'role': 'system', 'content': system_message}]
    messages.extend(updated_history)
    messages.append({'role': 'user', 'content': message})

    response = gemini.chat.completions.create(model=MODEL, messages=messages, tools=tools)

    # if the finish reason is tool calls then, we need to call the tool
    if response.choices[0].finish_reason=="tool_calls":
        message = response.choices[0].message
        response = handle_tool_calls(message)
        messages.append(message)
        messages.append(response)

        for message in messages:
            print(message)
            
        response = gemini.chat.completions.create(model=MODEL, messages=messages)
    
    return response.choices[0].message.content

In [40]:
def handle_tool_calls(message):
    response = []
    for tool_call in message.tool_calls:
        if tool_call.function.name == "get_ticket_price":
            arguments = json.loads(tool_call.function.arguments)
            city = arguments.get('destination_city')
            price_details = get_ticket_price(city)
            response.append({
                "role": "tool",
                "content": price_details,
                "tool_call_id": tool_call.id
            })
    return response

In [ ]:
gr.ChatInterface(fn=chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7897
* To create a public link, set `share=True` in `launch()`.


Tool called for city Berlin
Tool called for city Paris
{'role': 'system', 'content': "\nYou are a helpful assistant, for an airline called FlightAI.\nGive short and courteous responses no more than one sentence long.\nAlways be accurate, if you don't know the answer, just say you don't know.\n"}
{'role': 'user', 'content': 'Hi there'}
{'role': 'assistant', 'content': 'Hello! How can I help you today?'}
{'role': 'user', 'content': 'Which flight is cheaper ? to Berlin or to Paris ?'}
ChatCompletionMessage(content=None, refusal=None, role='assistant', annotations=None, audio=None, function_call=None, tool_calls=[ChatCompletionMessageFunctionToolCall(id='function-call-11644138011882655353', function=Function(arguments='{"destination_city":"Berlin"}', name='get_ticket_price'), type='function'), ChatCompletionMessageFunctionToolCall(id='function-call-11644138011882653754', function=Function(arguments='{"destination_city":"Paris"}', name='get_ticket_price'), type='function')])
[{'role': 'tool